## Lecture CSV

In [13]:
import plotly.express as px
import pandas as pd

In [14]:
# Lecture du CSV
df_sd = pd.read_csv("../datas/SpeedDatingData.csv", encoding = "ISO-8859-1")
df_sd.head()

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,...,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
0,1,1.0,0,1,1,1,10,7,NaN,4,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
1,1,1.0,0,1,1,1,10,7,NaN,3,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
2,1,1.0,0,1,1,1,10,7,NaN,10,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
3,1,1.0,0,1,1,1,10,7,NaN,5,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
4,1,1.0,0,1,1,1,10,7,NaN,7,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN


---

## Analyse - Axe 3

### Partager des intérêts communs, est-ce plus important que de paratger une origine éthnique ?

In [15]:
# Sélection des colonnes pertinentes pour l'analyse : match + int_corr (corrélation entre les évaluations des intérêts des participants et des partenaires) + samerace
columns_to_keep = ["match", "int_corr","samerace"]

# Vérifier les valeurs manquantes
missing_values = df_sd[columns_to_keep].isnull().sum()
print("Valeurs manquantes : ")
print(missing_values)

print("")

print("Pourcentage de valeurs manquantes : ")
print(100 * missing_values / df_sd.shape[0])

Valeurs manquantes : 
match         0
int_corr    158
samerace      0
dtype: int64

Pourcentage de valeurs manquantes : 
match       0.000000
int_corr    1.885892
samerace    0.000000
dtype: float64


J'ai peu de valeurs manquantes, je peux donc les supprimer.

In [16]:
print("Nombre de lignes avant suppression : {}".format(df_sd[columns_to_keep].shape[0]))

# Suppression des lignes avec des valeurs manquantes
df_sd_clean = df_sd[columns_to_keep].dropna()

print("Nombre de lignes après suppression : {}".format(df_sd_clean.shape[0]))

# Vérifier que les valeurs manquantes ont été remplacées
df_sd_clean.isnull().sum()

Nombre de lignes avant suppression : 8378
Nombre de lignes après suppression : 8220


match       0
int_corr    0
samerace    0
dtype: int64

---

#### Distribution de l'attribut `int_corr`

In [17]:
fig = px.histogram(df_sd_clean, x="int_corr", labels={"int_corr": "Correlation des intérêts"}, color_discrete_sequence=px.colors.qualitative.Pastel)
fig.update_layout(yaxis_title="Nombre d'individus", title="Distribution de la corrélation des intérêts entre participants", title_x=0.5)
fig.show()

Il s'agit d'un coefficient entre -1 et 1. 1 : intérêts alignés / 0 : neutre / -1 : intérêts opposés.

La corrélation des intérêts est princiaplement au-dessus de 0.

#### Distribution de la corrélation des intérêts selon un match

In [18]:
df_sd_clean["match_label"] = df_sd_clean["match"].map({0: "Non", 1: "Oui"})

In [19]:
fig = px.box(df_sd_clean, x="match_label", y="int_corr", color="match_label",
             labels={"match_label": "Match", "int_corr": "Corrélation d'intérêts"}, color_discrete_sequence=px.colors.qualitative.Pastel)
fig.update_layout(title="Distribution de la corrélation d'intérêts selon match", title_x=0.5)
fig.show()

Analyse

Sur ce boxplot on observe que les médianes et la répartition des corrélations sont très proches entre les deux groupes.

Cela signifie que la corrélation d'intérêts ne diffère pas beaucoup entre les cas où il y a match et ceux où il n'y en a pas. 

En d'autres termes, cette métrique ne semble pas être un facteur discriminant fort entre les deux catégories.

---

#### Répartition de l'attribut `samerace`

In [20]:
df_sd_clean["samerace"] = df_sd_clean["samerace"].map({0: "Non", 1: "Oui"})

In [21]:
df_samerace = df_sd_clean["samerace"].value_counts().reset_index()
display(df_samerace)

,samerace,count
0,Non,4924
1,Oui,3296


In [22]:
fig = px.pie(df_samerace, names = "samerace", values="count", color_discrete_sequence=px.colors.qualitative.Pastel)
fig.update_layout(title="Répartition des participants ayant la même origine éthnique", title_x=0.5)
fig.show()

#### Influence d'une même origine ethnique sur un match

In [23]:
# Calculer la proportion de matchs selon samerace
df_grouped = df_sd_clean.groupby("samerace")["match"].mean().reset_index()

fig = px.bar(df_grouped, x="samerace", y="match",
             labels={"samerace": "Même origine ethnique", "match": "Taux de match"},
             text="match", color_discrete_sequence=px.colors.qualitative.Pastel)
fig.update_layout(title="Taux de match selon l'origine ethnique", title_x=0.5)
fig.update_traces(texttemplate="%{text:.2%}")
fig.show()

Analyse

Sur ce barplot on observe que les taux de match sont très proches entre les deux groupes.

Cela signifie que les matchs ne diffèrent pas beaucoup entre les cas où les participants sont de même origine ethnique ou non. 

En d'autres termes, cette métrique ne semble pas être un facteur discriminant fort.

---

Nous pouvons donc conclure que partager des intérêts communs n'est pas plus important que de paratger une origine éthnique pour susciter l’intérêt d’une autre personne.

Ces deux facteurs ont en réalité très peu d’influence sur les matchs.